## 항공가격 배치 스크래핑
- 크롤링 배치 처리 파일 스크래핑 처리
    * 전체 파일 스크래핑 처리
    * 미처리 파일 오류 처리 3회 실시
    * 스크래핑 처리 파일 DB 처리

In [10]:
from collections import namedtuple
from datetime import datetime, timedelta
import requests
import sqlite3
import csv, io, os
from common.env_variable import *
from common.batch_util import *
from common.scrap_func import *
from common.crawl_func import *
from common.log_util import *
from common.util import save_raw_data
## 로그 초기화
logger_initialize('scrap_logger_setting.json')
init_env_variable('common/env_variable.json')

In [11]:
## 국내선 국제선 구분하여 폴더에 있는 파일 scraping 처리
## 포맷 : 대상항공사코드,플라이트,출발일,출발지,도착지,출발시간,도착시간,가격,유류세,세금,좌석수
## 오류 발생한 파일들은 해당 폴더에 그대로 방치, 주기적으로 체크하여 오래 남아 있는 파일들은 문제있는 파일로 취급 하여 재처리
def batch_scrap(crawl_dir,scrap_data_dir,dom_int=None):
    start_time = datetime.today()
    log_msgs = ['start batch job','scraping crawled file']
    log(log_msgs,level=logging.INFO)
    scrap_cnt = 0
    ## 스크랩 대상 파일(파일명, 내용)
    crawl_dir = crawl_dir
    files = get_crawled_file_list(crawl_dir)
    for file in files:
        log(file,level=logging.DEBUG)
        scrap_file = crawl_dir+'/'+file
        head, raw_data = read_crawled_file(scrap_file)
        ## 헤더 정보 읽어오기
        #head = get_headinfo(raw_data.splitlines()[0])
        log([head['dom_int'],head['site_code']],level=logging.DEBUG)
        if dom_int is not None and head['dom_int'] != dom_int: ## 국내선/국제선 체크
            continue
        site_code = head['site_code']
        ## 해당 함수 생성
        func = get_scrap_site_func(head['dom_int'],head['site_code'])
        log(['check func',func],level=logging.DEBUG)
        if func is None: ## 해당 함수가 없을 경우 다음으로
            continue
        try:
            scraped_list = func(head,raw_data)#''.join(raw_data.splitlines()[1:])))
        except:
            log_msgs = ['Error occured when doing scrap func -',func]
            log(log_msgs,level=logging.ERROR)
            continue
            
        ## 스크래핑 실패시 해당 파일 폴더에 보관, 해당 폴더 모니터링시 장기 미처리 파일 체크하여 처리
        if scraped_list is None:
            ## 출렬 처리
            log('Error occured in {}!'.format(file),level=logging.DEBUG)
            continue
        ## CSV 처리
        scraped_list_to_csv(set_headinfo(**head),scraped_list,scrap_data_dir+"/"+file.split(".")[0]+".csv")
        ## 처리 파일 이동 처리
        ## 이동 처리시 이미 파일이 존재하는 경우 삭제후 처리
        if os.path.exists(os.path.join(SCRAP_OK_DIR,os.path.split(scrap_file)[-1])):
            os.remove(os.path.join(SCRAP_OK_DIR,os.path.split(scrap_file)[-1]))
        move_scraped_file(scrap_file,SCRAP_OK_DIR)
        scrap_cnt += 1

    end_time = datetime.today()
    log_msgs = ['end batch job','scraping crawled file',
                'elapsed -{}'.format(end_time-start_time),'Total crawl:{} files saved.'.format(scrap_cnt)]
    log(log_msgs,level=logging.INFO)
## 에러 파일 추가 크롤링 처리
def batch_error(crawl_dir):
    ## 에러 파일 추가 크롤링 처리
    log('start recrawling batch job for error files')
    files = get_crawled_file_list(crawl_dir)
    cnt = 0
    for file in files:
        log(['error file',file])
        head,_=read_crawled_file(crawl_dir+'/'+file)
        func,isairline=get_crawl_site_func(head['dom_int'],head['site_code'])
        if isairline:
            raw_data = crawling_func(func,head['dpt'],head['arr'],head['dpt_date'])
            new_file = file_name(head['site_code'],head['dpt'],head['arr'],head['dpt_date'])
            new_head = set_headinfo(head['site_code'],head['dom_int'],head['site_code'],head['dpt'],head['arr'],
                                head['dpt_date'],crawl_date=head['crawl_date'])
        else:
            raw_data = crawling_func(func,head['airline'],head['dpt'],head['arr'],head['dpt_date'])
            new_file = file_name(head['site_code'],head['dpt'],head['arr'],head['dpt_date'],airline=head['airline'])
            new_head = set_headinfo(head['site_code'],head['dom_int'],head['airline'],head['dpt'],head['arr'],
                                head['dpt_date'],crawl_date=head['crawl_date'])
        ## 처리 파일 이동 : 해당 폴더의 error 폴더로 이동 처리
        ## 파일 명이 중복되는 경우 발생 가능, 이동 처리후 저장
        move_scraped_file(crawl_dir+'/'+file,crawl_dir+'/error')
        save_raw_data(new_file,raw_data,head=new_head)
        cnt += 1
    log(['end recrawling batch job','total {} files saved!'.format(cnt)])
## 파일 배치 처리
def batch_db(scrap_dir,db_ok_dir,db):
    start_time = datetime.today()
    log_msgs = ['start batch job','doing db job with scraped data file']
    log(log_msgs,level=logging.INFO)
    ## 파일 리스트 생성
    db_cnt = 0
    files = get_crawled_file_list(scrap_dir)
    ## 각 파일 처리
    for file in files:
        log(file,level=logging.DEBUG)
        csv_file = scrap_dir+'/'+file
        head, raw_data = read_crawled_file(csv_file,csv=True)
        ## 스키마 체크 - 차후 오류 로그 체크하여 해당 내용 확인후 반영
        ## DB 처리
        log('execute insert query',level=logging.DEBUG)
        i_cnt = scraped_csv_to_db(head,raw_data,db)
        log(['insert result',i_cnt],level=logging.DEBUG)
        db_cnt += i_cnt
        ## 정상 처리 파일 처리
        if i_cnt > 0:
            move_scraped_file(csv_file,db_ok_dir)
    end_time = datetime.today()
    log_msgs = ['end batch job','doing db job with scraped data file',
                'elapsed -{}'.format(end_time-start_time),'Total :{} rows inserted.'.format(db_cnt)]
    log(log_msgs,level=logging.INFO)
    
def scraped_csv_to_db(head,raw_data,db):
    log('$$ check csv to db process.',level=logging.INFO)
    ## CSV 파일 읽기
    csv_data = csv.reader(io.StringIO(raw_data))
    ## DB 처리 리스트 생성
    target_list = []
    for d in csv_data:
        ## scrap_date, scrap_site, patten(1 편도)
        td_list = [head['crawl_date'],head['site_code'],'1']
        ## airline,flt,dpt,arr,dpt_time,arr_time,fare,tax1,tax2,seat
        td_list.extend(d)
        target_list.append(td_list)
    ## DB 처리
    cnt = 0
    conn = sqlite3.connect(db)
    try:
        cur = conn.cursor()
        sql = "insert into airfare_scraped_data"+\
        "(scrap_date,scrap_site,patten,airline,flt,dpt_date,dpt,arr,dpt_time,arr_time,fare,tax1,tax2,seat)"+\
        "values(?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
        cur.executemany(sql,target_list)
        conn.commit()
        cnt = len(target_list)
    except sqlite3.Error as e:
        if conn:
            conn.rollback()
        log(e,level=logging.ERROR)
        cnt = 0
    finally:
        if conn:
            conn.close()
    log('$$ check db process result : {}'.format(cnt),level=logging.INFO)
    return cnt

## 스크랩 처리후 남은 파일 처리 - 필요없는 파일 nodata 폴더로 이동
## csv 파일 삭제 처리
def move_nodata_files():
    src = CRAWL_DIR
    dst = NODATA_DIR
    print('start moving error files')
    nodata_files = get_files(src,check='scrap')
    print('nodata files : ',len(nodata_files))
    for f in nodata_files:
        move_file(os.path.join('crawl', f),dst)
    print('end moving error files')
    ## csv 파일 데이터 삭제 처리
    for file in get_files(SCRAP_DATA_DIR):
        os.remove('scrap_data/'+file)

In [3]:
dom_int = None # 0 - 국내선, 1 - 국제선, None - 모두
batch_scrap(CRAWL_DIR,SCRAP_DATA_DIR,dom_int=dom_int)

batch_cnt = 3
db_file = 'airfare_scraped_data.db'
for i in range(batch_cnt):
    batch_error(CRAWL_DIR)
    batch_scrap(CRAWL_DIR,SCRAP_DATA_DIR,dom_int=dom_int)
batch_db(SCRAP_DATA_DIR,DB_OK_DIR,db_file)

2017-08-11 08:54:43,591 root     INFO     start batch job :: scraping crawled file
2017-08-11 08:54:43,624 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170812.txt
2017-08-11 08:54:43,627 root     DEBUG    1 :: 7C
2017-08-11 08:54:43,633 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:43,638 root     DEBUG    check func :: <function scraping_7C_int at 0x0000000007C727B8>
2017-08-11 08:54:43,652 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170813.txt
2017-08-11 08:54:43,657 root     DEBUG    1 :: 7C
2017-08-11 08:54:43,667 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:43,671 root     DEBUG    check func :: <function scraping_7C_int at 0x0000000007C727B8>
2017-08-11 08:54:43,696 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170814.txt
2017-08-11 08:54:43,702 root     DEBUG    1 :: 7C
2017-08-11 08:54:43,710 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:43,716 root     DEBUG    check func :: <function scraping_7C_int at 0x00000

start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data


2017-08-11 08:54:43,843 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170818.txt
2017-08-11 08:54:43,850 root     DEBUG    1 :: 7C
2017-08-11 08:54:43,858 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:43,863 root     DEBUG    check func :: <function scraping_7C_int at 0x0000000007C727B8>
2017-08-11 08:54:43,879 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170819.txt
2017-08-11 08:54:43,884 root     DEBUG    1 :: 7C
2017-08-11 08:54:43,887 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:43,894 root     DEBUG    check func :: <function scraping_7C_int at 0x0000000007C727B8>
2017-08-11 08:54:43,919 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170820.txt
2017-08-11 08:54:43,928 root     DEBUG    1 :: 7C
2017-08-11 08:54:43,939 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:43,946 root     DEBUG    check func :: <function scraping_7C_int at 0x0000000007C727B8>
2017-08-11 08:54:43,970 root     DEBUG    scrap_2017081108_7C_7C_BKK_I

start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data
start scraping jejuair crawled data
end scraping jejuair crawled data


2017-08-11 08:54:44,070 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:44,074 root     DEBUG    check func :: <function scraping_7C_int at 0x0000000007C727B8>
2017-08-11 08:54:44,087 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170824.txt
2017-08-11 08:54:44,099 root     DEBUG    1 :: 7C
2017-08-11 08:54:44,114 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:44,125 root     DEBUG    check func :: <function scraping_7C_int at 0x0000000007C727B8>
2017-08-11 08:54:44,143 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170825.txt
2017-08-11 08:54:44,153 root     DEBUG    1 :: 7C
2017-08-11 08:54:44,160 root     DEBUG    1 :: 7C :: scraping_7C_int
2017-08-11 08:54:44,164 root     DEBUG    check func :: <function scraping_7C_int at 0x0000000007C727B8>
2017-08-11 08:54:44,187 root     DEBUG    scrap_2017081108_7C_7C_BKK_ICN_20170826.txt
2017-08-11 08:54:44,191 root     DEBUG    1 :: 7C
2017-08-11 08:54:44,208 root     DEBUG    1 :: 7C :: scraping_7C_int
2

...

2017-08-11 09:01:01,845 root     DEBUG    error file :: scrap_2017081108_IP_ZE_PUS_GMP_20170901.txt
2017-08-11 09:01:01,848 root     DEBUG    0 :: IP :: crawling_IP_dom :: False
2017-08-11 09:01:01,851 root     DEBUG    Crawling Interpark domastic schedule site


End Simple crawling
Start Simple crawling :  http://domair.interpark.com/api/booking/airJourney.do
>> Parameters
inf:0 , tripDivi:0 , adt:1 , arr:GMP , depDate:20170901 , format:json , chd:0 , dep:PUS , airlineCode:ZE , 


2017-08-11 09:01:02,867 root     DEBUG    end recrawling batch job :: total 55 files saved!
2017-08-11 09:01:02,871 root     INFO     start batch job :: scraping crawled file
2017-08-11 09:01:02,878 root     DEBUG    scrap_2017081108_IP_7C_CJJ_CJU_20170812.txt
2017-08-11 09:01:02,887 root     DEBUG    0 :: IP
2017-08-11 09:01:02,891 root     DEBUG    0 :: IP :: scraping_IP_dom
2017-08-11 09:01:02,896 root     DEBUG    check func :: <function scraping_IP_dom at 0x0000000006984A60>
2017-08-11 09:01:02,911 root     DEBUG    start scraping Interpark crawled data
2017-08-11 09:01:02,918 root     WARNING  scraping data not found!
2017-08-11 09:01:02,923 root     DEBUG    Error occured in scrap_2017081108_IP_7C_CJJ_CJU_20170812.txt!
2017-08-11 09:01:02,931 root     DEBUG    scrap_2017081108_IP_7C_CJU_CJJ_20170815.txt
2017-08-11 09:01:02,940 root     DEBUG    0 :: IP
2017-08-11 09:01:02,946 root     DEBUG    0 :: IP :: scraping_IP_dom
2017-08-11 09:01:02,954 root     DEBUG    check func :: <fu

End Simple crawling


2017-08-11 09:01:03,051 root     WARNING  scraping data not found!
2017-08-11 09:01:03,058 root     DEBUG    Error occured in scrap_2017081108_IP_7C_GMP_CJU_20171001.txt!
2017-08-11 09:01:03,063 root     DEBUG    scrap_2017081108_IP_7C_GMP_CJU_20171002.txt
2017-08-11 09:01:03,070 root     DEBUG    0 :: IP
2017-08-11 09:01:03,079 root     DEBUG    0 :: IP :: scraping_IP_dom
2017-08-11 09:01:03,083 root     DEBUG    check func :: <function scraping_IP_dom at 0x0000000006984A60>
2017-08-11 09:01:03,088 root     DEBUG    start scraping Interpark crawled data
2017-08-11 09:01:03,094 root     WARNING  scraping data not found!
2017-08-11 09:01:03,099 root     DEBUG    Error occured in scrap_2017081108_IP_7C_GMP_CJU_20171002.txt!
2017-08-11 09:01:03,109 root     DEBUG    scrap_2017081108_IP_7C_GMP_CJU_20171003.txt
2017-08-11 09:01:03,115 root     DEBUG    0 :: IP
2017-08-11 09:01:03,120 root     DEBUG    0 :: IP :: scraping_IP_dom
2017-08-11 09:01:03,125 root     DEBUG    check func :: <functi

In [ ]:
## 추가 오류 처리 진행시
dom_int = None
db_file = 'airfare_scraped_data.db'
batch_error(CRAWL_DIR)
batch_scrap(CRAWL_DIR,SCRAP_DATA_DIR,dom_int=dom_int)
batch_db(SCRAP_DATA_DIR,DB_OK_DIR,db_file)

In [ ]:
## 스크랩 잔존 파일 이동 처리
move_nodata_files()

In [ ]:
db_file = 'airfare_scraped_data.db'
batch_db(SCRAP_DATA_DIR,DB_OK_DIR,db_file)